### Value Versus Referential Equality

There are two kinds of **equality**:  
- `Value equality` : Two **values** are equivalent in some sense
- `Referential equality` : Two **references** refer to `exactly the same object`.

- **Value types** use `value equality`.
- **Reference types** use `referential equality`. (This is **overridden** with `anonymous types` and `records`.)

In [ ]:
int x = 5, y = 5;
Console.WriteLine (x == y); // True (by virtue of value equality)

var dt1 = new DateTimeOffset (2010, 1, 1, 1, 1, 1, TimeSpan.FromHours(8));
var dt2 = new DateTimeOffset (2010, 1, 1, 2, 1, 1, TimeSpan.FromHours(9));
Console.WriteLine (dt1 == dt2); // True

In [2]:
class Foo { public int X; }

Foo f1 = new Foo { X = 5 };
Foo f2 = new Foo { X = 5 };
Console.WriteLine (f1 == f2); // False

Foo f3 = f1;
Console.WriteLine (f1 == f3); // True

False


In [ ]:
//you can customize reference types to exhibit value equality.
Uri uri1 = new Uri ("http://www.linqpad.net");
Uri uri2 = new Uri ("http://www.linqpad.net");
Console.WriteLine (uri1 == uri2); // True


var s1 = "http://www.linqpad.net";
var s2 = "http://" + "www.linqpad.net";
Console.WriteLine (s1 == s2); // True

### Standard Equality Protocols

There are `three standard protocols` that **types** can **implement** for `equality comparison`  
- The `==` and `!=` operators
- The **virtual** `Equals` method in **object**
- The `IEquatable<T>` interface

#### `==` and `!=`

they are ***operators***; thus, they are `statically` **resolved** (in fact, they are implemented as `static functions`).

when you **use** `==` or `!=`, C# makes a `compile-time decision` as to which type(`value` or `refrential` equality)will perform the **comparison**

In [ ]:
int x = 5;
int y = 5;
Console.WriteLine (x == y); // True 
//by value equality

object x1 = 5;
object y1 = 5;
Console.WriteLine (x1 == y1); // False
//by refretial equality

#### The virtual Object.Equals method

`Equals` is defined in `System.Object` and so is ***available*** to `all types`.   
**Equals** is resolved at `runtime` according to the object’s ***actual type***.

In [ ]:
object x = 5;
object y = 5;
Console.WriteLine (x.Equals (y)); // True
//by value equality , at the runtime

With ***reference types***, `Equals` performs `referential equality comparison` by default;  
  
with ***structs***, `Equals` performs `structural comparison` by calling `Equals` on ***each of its fields***.

The ***question that arises*** is why **designers of C#** didn’t avoid the problem by **making** `==` **virtual** and thus functionally `identical to Equals`  
There are three reasons for this  
- If the **first operand** is `null`, ***Equals*** fails with a `NullReferenceException`; a static operator does not.
- Because the `==` operator is `statically resolved`, it executes extremely `quickly`.
- Sometimes it can be `useful` to have == and Equals apply `different definitions` of equality.

In [ ]:
//consider the following example
public static bool AreEqual (object obj1, object obj2)
    => obj1.Equals (obj2);

//if the first argument is null, you get NullReferenceException.
//so we can fixed with this example
public static bool AreEqual1 (object obj1, object obj2)
{
    if (obj1 == null) return obj2 == null;
    return obj1.Equals (obj2);
}

//or
public static bool AreEqual3 (object obj1, object obj2)
    => obj1 == null ? obj2 == null : obj1.Equals (obj2);

##### The static object.Equals method

The `object class` provides a **static helper** method that does the work of `AreEqual`, Its name is `Equals`

In [ ]:
public static bool Equals (object objA, object objB);
//provides a null-safe equality comparison algorithm

In [ ]:
object x = 3, y = 3;
Console.WriteLine (object.Equals (x, y)); // True
x = null;
Console.WriteLine (object.Equals (x, y)); // False
y = null;
Console.WriteLine (object.Equals (x, y)); // True

In [ ]:
class Test <T>
{
    T _value;
    public void SetValue (T newValue)
    {
        if (!object.Equals (newValue, _value))
        {
            _value = newValue;
            OnValueChanged();
        }
    }
    protected virtual void OnValueChanged() { /*logic here*/ }
}

##### The static object.ReferenceEquals method

In [ ]:
class Widget { }
Widget w1 = new Widget();
Widget w2 = new Widget();
Console.WriteLine (object.ReferenceEquals (w1, w2)); // False
//check ReferenceEquals

### The IEquatable<T> interface

A consequence(نتیجه) of calling `object.Equals` is that it `forces boxing` on ***value types***. This is ***undesirable(نامطلوب)*** in `highly performance` ***sensitive scenarios***

In [ ]:
//A solution was introduced in C# 2.0, with the IEquatable<T> interface:
public interface IEquatable<T>
{
    bool Equals (T other);
}

when implemented, gives the `same result` as calling object’s virtual `Equals` method but `more quickly`.

In [ ]:
//Most basic .NET types implement IEquatable<T>.
//You can use IEquatable<T> as a constraint in a generic type

class Test<T> where T : IEquatable<T>
{
    public bool IsEqual (T a, T b)
    {
        return a.Equals (b); // No boxing with generic T
    }
}

### Equality and Custom Types

Recall default equality comparison behavior:
- ***Value types*** use `value equality`.
- ***Reference types*** use `referential equality` ***unless overridden*** (as is the case with
anonymous types and records).

Sometimes, it makes sense to ***override this behavior*** when `writing a type`. There are two cases for doing so:
- To **change** the `meaning of equality`
- To **speed up** `equality comparisons` for ***structs***

#### Changing the meaning of equality

Maybe the ***default behavior*** of `==` and `Equals` is **unnatural** for your type and is not what a `consumer would expect`. An example is `DateTimeOffset`

With ***classes***, it’s sometimes more ***natural*** to offer `value equality` as the ***default*** *instead* of `referential equality`. An example is `System.Uri`

With **records**, the ***compiler*** automatically implements `structural equality`. Sometimes, however, this will include **fields** that `you don’t want to compare`

#### Speeding up equality comparisons with structs

The default **structural equality comparison** algorithm for `structs` is `relatively slow`.  
  
Taking over this process by ***overriding Equals*** can **improve performance** by a
`factor of five`.  
  
**Overloading** the `==` operator and `implementing IEquatable<T>` allows unboxed equality comparisons

### How to override equality semantics

To `override equality` with **classes** or *structs**, here are the steps:
- ***Override*** `GetHashCode()` and `Equals()`.
- (Optionally) **overload** `!=` and `==.`
- (Optionally) **implement** `IEquatable<T>`.

In [4]:
public class PersonWithoutEqualOverride
{
    public PersonWithoutEqualOverride(string firstName, string lastName)
    {
        FirstName = firstName;
        LastName = lastName;
    }
    public string FirstName { get; }
    public string LastName { get; }
}

var p1 = new PersonWithoutEqualOverride( "kaveh","ahangar");
var p2 = new PersonWithoutEqualOverride( "kaveh","ahangar");
Console.WriteLine(p1.Equals(p2));

var dictionary = new Dictionary<PersonWithoutEqualOverride, string>(); 
dictionary[p1] = "Developer";
dictionary[p2] = "Designer";

Console.WriteLine(dictionary[p1]); // Output: Developer
Console.WriteLine(dictionary[p2]); // Output: Designer


False
Developer
Designer


In [6]:

public class Person
{
    public string FirstName { get; }
    public string LastName { get; }

    public Person(string firstName, string lastName)
    {
        FirstName = firstName;
        LastName = lastName;
    }

    // Override Equals
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }

        Person other = (Person)obj;
        return FirstName == other.FirstName && LastName == other.LastName;
    }

    // Override GetHashCode
    public override int GetHashCode()
    {
        return HashCode.Combine(FirstName, LastName);
    }
}

var p3 = new Person( "kaveh","ahangar");
var p4 = new Person( "kaveh","ahangar"); 
Console.WriteLine(p3.Equals(p4));


var dictionary1 = new Dictionary<Person, string>(); 
dictionary1[p3] = "Developer";
dictionary1[p4] = "Designer";

Console.WriteLine(dictionary1[p3]);
Console.WriteLine(dictionary1[p4]);

Console.WriteLine(p3 == p4);

True
Designer
Designer
False


In [1]:
int x = 5;
Console.WriteLine(x.GetHashCode());

5


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h5>متد <code>GetHashCode</code></h5>
<ol>
<li>
<p><strong>کاربرد و اهمیت <code>GetHashCode</code>:</strong></p>
<ul>
<li>متد <code>GetHashCode</code> در کلاس پایه <code>System.Object</code> تعریف شده است و هدف اصلی آن استفاده در انواع داده‌ای مانند <code>System.Collections.Hashtable</code> و <code>System.Collections.Generic.Dictionary&lt;TKey, TValue&gt;</code> است.</li>
<li>این متد یک عدد <code>Int32</code> به عنوان هش کد تولید می‌کند که برای تخصیص کارآمد عناصر بر اساس کلیدهایشان در هش‌جدول‌ها استفاده می‌شود.
</li>
</ul>
</li>
<li>
<p><strong>لزوم همزمانی با <code>Equals</code>:</strong></p>
<ul>
<li>زمانی که متد <code>Equals</code> را در یک کلاس یا ساختار بازنویسی می‌کنید، باید متد <code>GetHashCode</code> را نیز بازنویسی کنید.
</li>
<li>دلیل این کار این است که اگر دو شیء با استفاده از <code>Equals</code> برابر باشند، باید هش کد مشابهی نیز داشته باشند.
</li>
</ul>
</li>
<li><p><strong>قوانین بازنویسی <code>GetHashCode</code>:</strong></p>
<ul>
<li>باید برای دو شیء که <code>Equals</code> آنها برابر است، مقدار یکسانی برگرداند.</li><li>نباید استثناها را پرتاب کند.
</li>
<li>باید مقدار یکسانی برگرداند اگر بر روی همان شیء چندین بار فراخوانی شود، مگر اینکه شیء تغییر کرده باشد.
</li>
</ul>
</li>
</ol>

<h5>بهبود کارایی هش‌جدول‌ها:</h5>
<ol><li><p><strong>تنوع هش کد:</strong></p><ul><li>برای بهبود کارایی هش‌جدول‌ها، متد <code>GetHashCode</code> باید طوری نوشته شود که احتمال برخورد (دو مقدار متفاوت با هش کد یکسان) را به حداقل برساند.</li></ul></li><li><p><strong>اجرای پیش‌فرض در ساختارها و کلاس‌ها:</strong></p><ul><li>پیاده‌سازی پیش‌فرض <code>GetHashCode</code> در ساختارها ممکن است بر اساس تمام فیلدهای ساختار باشد.</li><li>در مقابل، پیاده‌سازی پیش‌فرض <code>GetHashCode</code> در کلاس‌ها بر اساس یک توکن داخلی شیء است که در نسخه فعلی CLR برای هر نمونه یکتا است.</li></ul></li></ol>

</div>

### Overloading `==` and `!=`

In [ ]:
var sb1 = new StringBuilder ("foo");
var sb2 = new StringBuilder ("foo");
Console.WriteLine (sb1 == sb2); // False (referential equality)
Console.WriteLine (sb1.Equals (sb2)); // True (value equality)

In [2]:
public class Person
{
    public string FirstName { get; }
    public string LastName { get; }

    public Person(string firstName, string lastName)
    {
        FirstName = firstName;
        LastName = lastName;
    }
    
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }

        Person other = (Person)obj;
        return FirstName == other.FirstName && LastName == other.LastName;
    }

    // Override GetHashCode
    public override int GetHashCode()
    {
        return HashCode.Combine(FirstName, LastName);
    }

    public static bool operator == (Person p1, Person p2) => p1.Equals(p2);

    public static bool operator != (Person p1, Person p2) => !p1.Equals(p2);
}

var p3 = new Person( "kaveh","ahangar");
var p4 = new Person( "kaveh","ahangar"); 
Console.WriteLine(p3.Equals(p4));


var dictionary1 = new Dictionary<Person, string>(); 
dictionary1[p3] = "Developer";
dictionary1[p4] = "Designer";

Console.WriteLine(dictionary1[p3]);
Console.WriteLine(dictionary1[p4]);

Console.WriteLine(p3 == p4);

True
Designer
Designer
True


### Implementing `IEquatable<T>`

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>پیاده‌سازی <code>IEquatable&lt;T&gt;</code> در مقایسه با تنها بازنویسی متد <code>Equals</code> از کلاس پایه <code>Object</code> مزایای مهمی دارد که در زیر به آن‌ها اشاره می‌شود:</p>

<h5>1. <strong>نوع امن‌تر و کارآمدتر:</strong></h5>

<ul><li>وقتی <code>IEquatable&lt;T&gt;</code> را پیاده‌سازی می‌کنید، یک متد <code>Equals</code> نوع-ایمن و خاص نوع (<code>T</code>) ارائه می‌دهید. این امر باعث می‌شود که از تبدیل‌های نوعی غیرضروری جلوگیری کنید.</li><li>متد <code>Equals(Object)</code> از کلاس پایه <code>Object</code> نیاز به تبدیل نوع به <code>T</code> دارد که ممکن است هزینه‌بر باشد و به صورت جعبه‌ای (boxing) در ساختارها منجر شود.</li></ul>

<h5>2. <strong>پشتیبانی بهتر از Generic Collections:</strong></h5>
<ul><li>بسیاری از مجموعه‌های جنریک مانند <code>List&lt;T&gt;</code>، <code>Dictionary&lt;TKey, TValue&gt;</code> و <code>HashSet&lt;T&gt;</code> از <code>IEquatable&lt;T&gt;</code> برای مقایسه عناصر استفاده می‌کنند. پیاده‌سازی این رابط به شما امکان می‌دهد که از این مجموعه‌ها به صورت بهینه‌تری استفاده کنید.</li><li>هنگامی که <code>IEquatable&lt;T&gt;</code> پیاده‌سازی می‌شود، متدهای جنریک <code>Equals</code> فراخوانی می‌شوند که کارآمدتر از فراخوانی نسخه عمومی <code>Equals(Object)</code> هستند.</li></ul>
<h5>3. <strong>کارایی و عملکرد بهتر:</strong></h5>
<ul><li>پیاده‌سازی <code>IEquatable&lt;T&gt;</code> به طور کلی کارایی بهتری دارد زیرا از تبدیل نوع و جعبه‌ای (boxing) جلوگیری می‌کند.</li><li>این موضوع به ویژه در مورد ساختارها (structs) اهمیت دارد، زیرا جعبه‌ای (boxing) می‌تواند به عملکرد آسیب بزند.</li></ul>
<h5>4. <strong>کد خواناتر و قابل فهم‌تر:</strong></h5>
<ul><li>با پیاده‌سازی <code>IEquatable&lt;T&gt;</code>, کد شما خواناتر و قابل فهم‌تر می‌شود، زیرا دیگر نیازی به تبدیل نوع و بررسی‌های نوعی اضافی نیست.</li><li>متد <code>Equals</code> پیاده‌سازی شده از طریق <code>IEquatable&lt;T&gt;</code>، فقط با نوع مورد نظر شما سر و کار دارد و واضح‌تر است.</li></ul>
</div>

In [ ]:
public class Person : IEquatable<Person>
{
    public string Name { get; set; }
    public int Age { get; set; }

    public bool Equals(Person other)
    {
        if (other == null) return false;
        return this.Name == other.Name && this.Age == other.Age;
    }

    /*
    برای همه سناریوها از جمله
     مجموعه‌های غیر جنریک و مواردی که با نوع پایه
      Object سروکار دارند، کار می‌کند.
    */
    public override bool Equals(object obj)
    {
        if (obj == null) return false;
        if (obj.GetType() != this.GetType()) return false;
        return Equals(obj as Person);
    }

    public override int GetHashCode()
    {
        return HashCode.Combine(Name, Age);
    }
}


In [ ]:
public struct Area : IEquatable<Area>
{
    public readonly int Measure1;
    public readonly int Measure2;
    
    public Area(int m1, int m2)
    {
        Measure1 = Math.Min(m1, m2);
        Measure2 = Math.Max(m1, m2);
    }
    public override bool Equals(object other)
        => other is Area a && Equals(a); // Calls method below

    public bool Equals(Area other) // Implements IEquatable<Area>
        => Measure1 == other.Measure1 && Measure2 == other.Measure2;

    public override int GetHashCode()
        => HashCode.Combine(Measure1, Measure2);

    public static bool operator ==(Area a1, Area a2) => a1.Equals(a2);

    public static bool operator !=(Area a1, Area a2) => !a1.Equals(a2);
}